In [1]:
import httplib2
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from parsel import Selector
from statistics import mean
import time
import requests
from datetime import datetime
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
http = httplib2.Http()

In [2]:
base_url = "https://www.prisjakt.nu"
url = "https://www.prisjakt.nu/c/tv"
page_links = []
links = []
page_links.append(url)
for page in page_links:
    url = page
    i = requests.get(url).text
    soup=BeautifulSoup(i,'html.parser')
    pages = soup.find_all("li",{"class":"PaginationListItem-sc-11oisbu-1 bKXcXk"})
    for page in pages:
        link = page.find("a", {"data-test":"PaginationLink"})
        links.append(link)
        if link == None:
            links.remove(link)
    for l in links:
        pg = l.get('href')
        full_url = base_url+pg
        while full_url not in page_links:
            page_links.append(full_url)


print(f'Pages Found: {len(page_links)}')

Pages Found: 47


In [3]:
productlists = []
for link in page_links:
        k = requests.get(link).text
        soup=BeautifulSoup(k,'html.parser')
        productlist = soup.find_all("article",{"data-test":"ProductGridCard"})
        productlists.append(productlist)

In [4]:
items = [item for sublist in productlists for item in sublist]

In [5]:
productlinks = []
for item in items:
        link = item.find("a",{"data-test":"InternalLink"}).get('href')                 
        productlinks.append(base_url+link)

print(f'Number of Products Found: {len(productlinks)}')

Number of Products Found: 2038


In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from parsel import Selector
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# configure webdriver
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [7]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.21M/6.21M [00:08<00:00, 750kB/s] 
C:\Users\user\AppData\Local\Temp\ipykernel_16960\2767233116.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [10]:
property_list = []
for link in productlinks:
    url = link+"#properties"
    driver.get(url)
    try:
        element1 = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//body/div[@id='root']/div[@class='App-sc-2fu3f8-0 ljZBoE']/section[@class='Main-sc-2fu3f8-1 hnCjSa']/div[@class='Content-sc-2fu3f8-2 hybPGh']/div[@class='PageContent-sc-1wgu331-5 dfQdce']/div[@class='Wrapper-sc-1wgu331-1 gPsMfp']/div[@class='Content-sc-1wgu331-0 gMPJqu']/div[@class='ProductPage-sc-mnx6sq-0 iMqgaO']/div/div[@id='#properties']/div[@class='PageSectionWrapper-sc-o2pbvx-0 kQdrfN']/section[@class='PageSectionContent-sc-o2pbvx-1 hbBYkf']/section[@data-test='PropertiesTabContent']/div[@class='StyledList--ji0zot McirA StyledPanel--17i5iqi ffQfPu']/div[@data-test-type='product-info']/div[@class='StyledViewport-sc-7zjdbj-0 iDfoDl']/section[1]/div[1]/div[1]"))
        )
        element1.click()
    except:
        element1 = None
    page_source =driver.page_source
    soup = BeautifulSoup(page_source,'html.parser')
    specs = soup.find_all("section", {"class":"Section-sc-1jnk5ag-1 kLbzuK"})
    prop_names = []
    prop_vals = []
    for sect in specs:
        sections = sect.find_all("div", {"class", "Row-sc-1jnk5ag-3 jQedgK"})
        for section in sections:
            properties = section.find_all("div", {"class":"Column-sc-1jnk5ag-2 bIOAXd"})
            for property in properties:
                property_name = property.find("span", {"class":"Text--bzqghn wmwsE bodysmalltext PropertyName-sc-1jnk5ag-5 hEdIXF"})
                property_value = property.find("span", {"class":"Text--bzqghn ddqBq bodysmalltext PropertyValue-sc-1jnk5ag-6 etcKHO"})
                if property_name !=None:
                    prop_names.append(property_name.text.replace('\n',""))
                elif property_value !=None:
                    prop_vals.append(property_value.text.replace('\n',""))
    properties = {}
    for name in prop_names:
        for val in prop_vals:
            properties[name] = val
            prop_vals.remove(val)
            break
    property_list.append(properties)


MaxRetryError: HTTPConnectionPool(host='localhost', port=53763): Max retries exceeded with url: /session/616d2d0dbfb39bac593714951c4449ea/timeouts (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001CD7C4ADA00>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:

def get_prices():
        page_source = driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        offers  = soup.find("ul", {"class":"PriceList-sc-wkzg9v-0 fbrkVc"})
        price_list = []
        for offer in offers:
            offer_prices  = offer.find_all("div", {"class":"PriceWrapper-sc-k40pbc-4 OYetQ"})
            price_list.append(offer_prices)

        price_list = [price for sublist in price_list for price in sublist]
        for i in range(0, len(price_list)):
            price_list[i] = price_list[i].text.replace('\n',"")

        for i in range(0, len(price_list)):
            price_list[i] = re.sub('[\xa0]', '',price_list[i])
            price_list[i] = re.sub('[kr]', '', price_list[i])
            price_list[i]= price_list[i].replace(r'[()]',"")

        for i in range(0, len(price_list)):
            price_list[i] = int(price_list[i])
        no_offers = len(price_list)
        avg_price =  mean(price_list)

        return no_offers, avg_price

In [ ]:
data = []

In [ ]:
product_scraped = 1
for link in productlinks:
    from datetime import datetime
    f = requests.get(link,headers=headers).text
    hun=BeautifulSoup(f,'html.parser')
    driver.get(link)
    try:
    #  click on cookie banner
        banner_click = driver.find_element(By.CSS_SELECTOR, "div.BannerCookieBanner div.BannerContentCookieBanner div.ButtonsWrapperCookieBanner.AcceptButtonWrapperCookieBanner button.AcceptButtonCookieBanner.ButtonCookieBanner span.TextWrapperCookieBanner > span:nth-child(1)")
        banner_click.click()
    except:
        banner_click = None
    try:
            #click to reveal more items
        reveal_click = driver.find_element(By.CSS_SELECTOR, "div.App-sc-2fu3f8-0.ljZBoE section.Main-sc-2fu3f8-1.hnCjSa:nth-child(5) div.Content-sc-2fu3f8-2.hybPGh div.PageContent-sc-1wgu331-5.dfQdce div.Wrapper-sc-1wgu331-1.gPsMfp div.Content-sc-1wgu331-0.gMPJqu div.ProductPage-sc-mnx6sq-0.iMqgaO div.SectionWrapper-sc-ia0zhw-0.bqrwQK:nth-child(1) div.PageSectionWrapper-sc-o2pbvx-0.gHOcYt.StyledPageSection-sc-1w3nqup-0.UUqee:nth-child(1) section.PageSectionContent-sc-o2pbvx-1.euuGdp section:nth-child(1) div.PriceListPanelWrapper-sc-1htx0wh-0.lkPiWw div.PricePanel-sc-wkzg9v-1.kSsiTe div:nth-child(3) > button.BaseButton--uexw3i.cVdzxy.textbutton")
        reveal_click.click()
    except:
        reveal_click = None
    
    def get_prices():
        page_source = driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        offers  = soup.find("ul", {"class":"PriceList-sc-wkzg9v-0 fbrkVc"})
        price_list = []
        for offer in offers:
            offer_prices  = offer.find_all("div", {"class":"PriceWrapper-sc-k40pbc-4 OYetQ"})
            price_list.append(offer_prices)

        price_list = [price for sublist in price_list for price in sublist]
        for i in range(0, len(price_list)):
            price_list[i] = price_list[i].text.replace('\n',"")

        for i in range(0, len(price_list)):
            price_list[i] = re.sub('[\xa0]', '',price_list[i])
                #price_list[i] = re.sub('[kr]', '', price_list[i])
            price_list[i] = re.sub("[^0-9]", "", price_list[i])
                #price_list[i]= price_list[i].replace(r'[()]',"")

        for i in range(0, len(price_list)):
            price_list[i] = int(price_list[i])
        no_offers = len(price_list)
        avg_price =  "{:.2f}".format(mean(price_list))

        return no_offers, avg_price

    try:
        category = "TV"
    except:
        category = None
    try:
        name = hun.find("div",{"data-test":"ProductTitle"}).text.replace('\n',"")
    except:
        name = None
    try:
        avg_price = get_prices()[1]
    except:
        avg_price = None
    try:
        offs = get_prices()[0]
    except:
        offs = None
    try:
        country = "NO"
    except:
        country = None
    try:
        currency = "NOK"
    except:
        currency = None
    try:
        rank = hun.find("span",{"class":"Text--bzqghn kBBoFI titlesmalltext StyledCaptionText--7mhtyj fxXtPM"}).text.replace('\n',"")
    except:
        rank = None
    try:
        rating = hun.find("span",{"class":"RateNumber-sc-14ktvqu-5 ifhFUL"}).text.replace('\n',"")
    except:
        rating = None
    try:
        reviews = hun.find("span",{"class":"Counter-sc-14ktvqu-0 bORxLl"}).text.replace('\n',"")
    except:
        reviews = None
    try:
        description = hun.find("p",{"class":"ProductInfo-sc-46oo5c-0 bRxlhS"}).text.replace('\n',"")
    except:
        description = None
    #try:
        #specifications = get_specs(prod_specifications)
    #except:
        #specifications = None

    try:
        scrape_link = link
        
    except:
        scrape_link = None
    
    try:
        now = datetime.now()
        date_scraped = now.strftime("%d/%m/%Y %H:%M:%S")
    except:
        date_scraped = None


    tv = {"Country Code":country, "Category":category, "Name":name, "Avg Price": avg_price, "Currency":currency, "Nb of Offers":offs, "Rank":rank, "Rating":rating, "Reviews":reviews, "Description":description, \
        "scrape_link":scrape_link, "date_scraped":date_scraped}
    
    data.append(tv)
    
    print(f'product scraped: {product_scraped, name}')
    product_scraped = product_scraped + 1

In [ ]:
df_prop = pd.DataFrame.from_dict(property_list)
df_prop

,Varumärke,Bildstorlek,Upplösning,Bildskärmsteknik,HDR,Smart TV,Lanseringsår,Pris/tum,Produktnamn,Kategori,...,Effektförbrukning drift (min-max),Effektförbrukning standby/viloläge,Energiförbrukning i HDR-läge (per 1000h),Energiförbrukning i SDR-läge (per 1000h),Energiklass (A-G),Energiklass (HDR),Bakbelysning,Diodplacering,Teknologier,Paneltyp
0,LG,65 tum,4K Ultra HD (3840x2160),OLED,Ja,Ja,2021,199.83 kr/tum,"LG OLED65C1 65"" 4K Ultra HD (3840x2160) OLED S...",TV,...,126 - 222 W,0.5 W,222 kWh,126 kWh,G,G,NaN,NaN,NaN,NaN
1,LG,55 tum,4K Ultra HD (3840x2160),OLED,Ja,Ja,2021,172.55 kr/tum,"LG OLED55C1 55"" 4K Ultra HD (3840x2160) OLED S...",TV,...,106 W,0.5 W,171 kWh,106 kWh,G,G,NaN,NaN,NaN,NaN
2,Philips,58 tum,4K Ultra HD (3840x2160),LCD,Ja,Ja,,103.28 kr/tum,"Philips 58PUS9006 58"" 4K Ultra HD (3840x2160) ...",TV,...,90 W,0.3 W,128 kWh,90 kWh,,G,LED (lysdiod),Bakbelyst (Back),,VA
3,LG,77 tum,4K Ultra HD (3840x2160),OLED,Ja,Ja,2021,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Samsung,50 tum,4K Ultra HD (3840x2160),LCD,Ja,Ja,2021,109.8 kr/tum,"Samsung QLED QE50Q60A 50"" 4K Ultra HD (3840x21...",TV,...,67 W,0.5 W,100 kWh,67 kWh,G,G,LED (lysdiod),Bakbelyst (Back),Quantum dot,
5,LG,55 tum,4K Ultra HD (3840x2160),OLED,Ja,Ja,2021,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Samsung,32 tum,Full HD (1920x1080),LCD,Ja,Ja,2020,113.56 kr/tum,"Samsung The Frame QE32LS03T 32"" Full HD (1920x...",TV,...,33 - 55 W,0.5 W,55 kWh,33 kWh,G,,LED (lysdiod),,Quantum dot,
7,LG,48 tum,4K Ultra HD (3840x2160),OLED,Ja,Ja,2021,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Samsung,65 tum,4K Ultra HD (3840x2160),LCD,Ja,Ja,2021,191.54 kr/tum,"Samsung QLED QE65QN85A 65"" 4K Ultra HD (3840x2...",TV,...,,0.5 W,193 kWh,101 kWh,F,G,LED (lysdiod),Bakbelyst (Back),"Quantum dot, Local Dimming, Mini-LED",IPS
9,Samsung,65 tum,4K Ultra HD (3840x2160),LCD,Ja,Ja,2021,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data

[{'Country Code': 'SW',
  'Category': 'TV',
  'Name': 'LG OLED65C1 65" 4K Ultra HD (3840x2160) OLED Smart TV',
  'Avg Price': 18722.166666666668,
  'Currency': 'SEK',
  'Nb of Offers': 18,
  'Rank': 'Rank 1',
  'Rating': '4.2',
  'Reviews': '(22 omdömen)',
  'Description': 'Det billigaste priset för LG OLED65C1 65" 4K Ultra HD (3840x2160) OLED Smart TV just nu är 12\xa0989\xa0kr. Det är den populäraste produkten i kategorin TV med ett genomsnittligt betyg på 4.2 av 5.  Prisjakt jämför priser och erbjudanden från 18 butiker. ',
  'scrape_link': 'https://www.prisjakt.nu/produkt.php?p=5732274',
  'date_scraped': '14/08/2022 11:22:54'},
 {'Country Code': 'SW',
  'Category': 'TV',
  'Name': 'LG OLED55C1 55" 4K Ultra HD (3840x2160) OLED Smart TV',
  'Avg Price': 13332.764705882353,
  'Currency': 'SEK',
  'Nb of Offers': 17,
  'Rank': 'Rank 2',
  'Rating': '4.1',
  'Reviews': '(3 omdömen)',
  'Description': 'Det billigaste priset för LG OLED55C1 55" 4K Ultra HD (3840x2160) OLED Smart TV just 

In [ ]:
df = pd.DataFrame.from_dict(data)
df.shape

(30, 12)

In [ ]:
df.columns

Index(['Country Code', 'Category', 'Name', 'Avg Price', 'Currency',
       'Nb of Offers', 'Rank', 'Rating', 'Reviews', 'Description',
       'scrape_link', 'date_scraped', 'Varumärke', 'Bildstorlek', 'Upplösning',
       'Bildskärmsteknik', 'HDR', 'Smart TV', 'Lanseringsår', 'Pris/tum',
       'Produktnamn', 'Kategori', 'TV-mottagare, digital', 'Dubbla mottagare',
       'Stöd för CI/CA-modul', 'CI+', 'Typ av digital-mottagare',
       'Analog 3,5mm-ingång (Aux)', 'Digital optisk ut (S/PDIF)', 'HDMI',
       'Antal HDMI-ingångar', 'HDCP-version', 'HDMI-version',
       'Standarder som stöds', 'Hörlursutgång', 'Komponent in (YPbPr/YCbCr)',
       'Komposit in', 'USB-kontakt', 'Inspelningsfunktion via USB (PVR)',
       'USB 2.0', 'USB 3.1 Gen 1 (5Gbps)', 'USB-kontakter (totalt)', 'VGA in',
       'Övriga anslutningar', 'Bildfrekvens', 'Bildförhållande',
       'Bildstorlek (cm)', 'Ljusstyrka', 'Rörelseförbättring (frekvens/index)',
       'Typ av index/mätmetod', 'Skärmtyp', 'Stödda vi

In [ ]:
df = pd.concat([df, df_prop.reindex(df.index)], axis=1)
#gen_spec_df = df["Generalspecifications"].apply(pd.Series)
#df = pd.concat([df, gen_spec_df.reindex(df.index)], axis=1)
df_c = df.copy()
# Remove specifications column
#df.drop(['specifications'], axis=1, inplace=True)
df

,Country Code,Category,Name,Avg Price,Currency,Nb of Offers,Rank,Rating,Reviews,Description,...,Effektförbrukning drift (min-max),Effektförbrukning standby/viloläge,Energiförbrukning i HDR-läge (per 1000h),Energiförbrukning i SDR-läge (per 1000h),Energiklass (A-G),Energiklass (HDR),Bakbelysning,Diodplacering,Teknologier,Paneltyp
0,SW,TV,"LG OLED65C1 65"" 4K Ultra HD (3840x2160) OLED S...",18722.166667,SEK,18,Rank 1,4.2,(22 omdömen),"Det billigaste priset för LG OLED65C1 65"" 4K U...",...,126 - 222 W,0.5 W,222 kWh,126 kWh,G,G,NaN,NaN,NaN,NaN
1,SW,TV,"LG OLED55C1 55"" 4K Ultra HD (3840x2160) OLED S...",13332.764706,SEK,17,Rank 2,4.1,(3 omdömen),"Det billigaste priset för LG OLED55C1 55"" 4K U...",...,106 W,0.5 W,171 kWh,106 kWh,G,G,NaN,NaN,NaN,NaN
2,SW,TV,"Philips 58PUS9006 58"" 4K Ultra HD (3840x2160) ...",10604.285714,SEK,7,Rank 3,3.7,(2 omdömen),Det billigaste priset för Philips 58PUS9006 58...,...,90 W,0.3 W,128 kWh,90 kWh,,G,LED (lysdiod),Bakbelyst (Back),,VA
3,SW,TV,"LG OLED77B1 77"" 4K Ultra HD (3840x2160) OLED S...",23323.333333,SEK,3,Rank 4,4.1,(4 omdömen),"Det billigaste priset för LG OLED77B1 77"" 4K U...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SW,TV,"Samsung QLED QE50Q60A 50"" 4K Ultra HD (3840x21...",7230.375000,SEK,16,Rank 5,None,None,Det billigaste priset för Samsung QLED QE50Q60...,...,67 W,0.5 W,100 kWh,67 kWh,G,G,LED (lysdiod),Bakbelyst (Back),Quantum dot,
5,SW,TV,"LG OLED55B1 55"" 4K Ultra HD (3840x2160) OLED S...",10709.444444,SEK,9,Rank 6,4.2,(5 omdömen),"Det billigaste priset för LG OLED55B1 55"" 4K U...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,SW,TV,"Samsung The Frame QE32LS03T 32"" Full HD (1920x...",4392.066667,SEK,15,Rank 7,None,None,Det billigaste priset för Samsung The Frame QE...,...,33 - 55 W,0.5 W,55 kWh,33 kWh,G,,LED (lysdiod),,Quantum dot,
7,SW,TV,"LG OLED48C1 48"" 4K Ultra HD (3840x2160) OLED S...",12048.307692,SEK,13,Rank 8,3.7,(4 omdömen),"Det billigaste priset för LG OLED48C1 48"" 4K U...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SW,TV,"Samsung QLED QE65QN85A 65"" 4K Ultra HD (3840x2...",15528.466667,SEK,15,Rank 9,3.8,(1 omdöme),Det billigaste priset för Samsung QLED QE65QN8...,...,,0.5 W,193 kWh,101 kWh,F,G,LED (lysdiod),Bakbelyst (Back),"Quantum dot, Local Dimming, Mini-LED",IPS
9,SW,TV,"Samsung QLED QE65QN90A 65"" 4K Ultra HD (3840x2...",18879.785714,SEK,14,Rank 10,3.5,(4 omdömen),Det billigaste priset för Samsung QLED QE65QN9...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#df2 = df.reset_index().rename(columns={'index':'id'})
df_long = df.set_index(['Country Code', 'Varumärke','Category', 'Name', 'Nb of Offers', 'Lanseringsår', 'Avg Price', 'Currency', 'scrape_link', 'date_scraped']).stack().reset_index()

NotImplementedError: > 1 ndim Categorical are not supported at this time

In [ ]:
df_long

,Country Code,Varumärke,Category,Name,Nb of Offers,Lanseringsår,Avg Price,Currency,scrape_link,date_scraped,level_10,0
0,SW,LG,TV,"LG OLED65C1 65"" 4K Ultra HD (3840x2160) OLED S...",18,2021,18722.166667,SEK,https://www.prisjakt.nu/produkt.php?p=5732274,14/08/2022 11:22:54,Rank,Rank 1
1,SW,LG,TV,"LG OLED65C1 65"" 4K Ultra HD (3840x2160) OLED S...",18,2021,18722.166667,SEK,https://www.prisjakt.nu/produkt.php?p=5732274,14/08/2022 11:22:54,Rating,4.2
2,SW,LG,TV,"LG OLED65C1 65"" 4K Ultra HD (3840x2160) OLED S...",18,2021,18722.166667,SEK,https://www.prisjakt.nu/produkt.php?p=5732274,14/08/2022 11:22:54,Reviews,(22 omdömen)
3,SW,LG,TV,"LG OLED65C1 65"" 4K Ultra HD (3840x2160) OLED S...",18,2021,18722.166667,SEK,https://www.prisjakt.nu/produkt.php?p=5732274,14/08/2022 11:22:54,Description,"Det billigaste priset för LG OLED65C1 65"" 4K U..."
4,SW,LG,TV,"LG OLED65C1 65"" 4K Ultra HD (3840x2160) OLED S...",18,2021,18722.166667,SEK,https://www.prisjakt.nu/produkt.php?p=5732274,14/08/2022 11:22:54,Bildstorlek,65 tum
...,...,...,...,...,...,...,...,...,...,...,...,...
2025,SW,TCL,TV,"TCL 98C735 98"" 4K Ultra HD (3840x2160) QLED Go...",7,2022,53420.000000,SEK,https://www.prisjakt.nu/produkt.php?p=6219595,14/08/2022 11:24:47,Energiklass (HDR),
2026,SW,TCL,TV,"TCL 98C735 98"" 4K Ultra HD (3840x2160) QLED Go...",7,2022,53420.000000,SEK,https://www.prisjakt.nu/produkt.php?p=6219595,14/08/2022 11:24:47,Bakbelysning,LED (lysdiod)
2027,SW,TCL,TV,"TCL 98C735 98"" 4K Ultra HD (3840x2160) QLED Go...",7,2022,53420.000000,SEK,https://www.prisjakt.nu/produkt.php?p=6219595,14/08/2022 11:24:47,Diodplacering,Bakbelyst (Back)
2028,SW,TCL,TV,"TCL 98C735 98"" 4K Ultra HD (3840x2160) QLED Go...",7,2022,53420.000000,SEK,https://www.prisjakt.nu/produkt.php?p=6219595,14/08/2022 11:24:47,Teknologier,"Quantum dot, Local Dimming"


In [ ]:
df.to_csv("data_30.csv")

    prod_specifications = hun.find_all("div",{"class":"Column-sc-1jnk5ag-2 bIOAXd"})
    def get_specs(prod_specifications):
        nam = []
        val = []
        for spec in prod_specifications:
            for prop in spec.find_all("span", {"class":"Text--bzqghn wmwsE bodysmalltext PropertyName-sc-1jnk5ag-5 hEdIXF"}):
                prop_n  = prop.get_text()
                nam.append(prop_n)
            for prop in spec.find_all("span", {"class":"Text--bzqghn ddqBq bodysmalltext PropertyValue-sc-1jnk5ag-6 etcKHO"}):
                prop_v  = prop.get_text()
                val.append(prop_v)
        specs = {nam[i]: val[i] for i in range(len(nam))}
        return(specs)

In [ ]:
df_long.to_csv('df_long.csv')